# Training LR models

In this notebook, we show how to train the LR models, make predictions and store the weights (for model interpretation). How to analyze the results, can be found in the figure repoducibility notebooks. 

Input for the functions can be downloaded from Zenodo

In [1]:
import time as tm
from pathlib import Path

import numpy as np
import pandas as pd
import h5py

import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from sklearn.model_selection import GroupKFold

### LR model

First, we create a pytorch lightning class for our model. 

In [2]:
class ElasticLinear(pl.LightningModule):
    def __init__(self, input_dim, output_dim, loss_fn, 
                 alpha=0.1, l1_ratio=0.5, lr=0.05):
        super().__init__()
        
        self.save_hyperparameters(ignore=['loss_fn'])
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.loss_fn = loss_fn

        self.lr = lr
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.linear = torch.nn.Linear(input_dim, output_dim)
        self.sigmoid = torch.nn.Sigmoid()
        
        self.train_log = []

    def forward(self, x):
        x = self.linear(x)
        return x
    
    def criterion(self, y_pred, y):
        l1_loss = self.l1_ratio * torch.norm(self.linear.weight, p=1)
        l2_loss = (1-self.l1_ratio) * torch.norm(self.linear.weight, p=2)
        loss = self.loss_fn(y_pred, y) + self.alpha*(l1_loss+l2_loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y, ind = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)
        
        self.log("loss", loss, on_epoch=True)
        self.train_log.append(loss.detach().numpy())
        return loss
    
    def predict_step(self, batch, batch_idx):
        x, y, ind = batch
        return self(x), y
    
    def test_step(self, batch, batch_idx):
        x, y, ind = batch
        y_pred = self(x)
        test_loss = self.criterion(y_pred, y)
        self.log("test_loss", test_loss)
        
    def validation_step(self, batch, batch_idx):
        x, y, ind = batch
        y_pred = self(x)
        val_loss = self.criterion(y_pred, y)
        self.log("val_loss", val_loss)


### Create a data module

Create a data module that uses the RBP data as input and PSI values as output. We ensure that the folds are the same as for the DL models.

In [3]:
class ExonDataModule(pl.LightningDataModule):
    def __init__(self, 
                 PSI_neur,
                 PSI_glia,
                 RBP,
                 out_dir_neur: str = 'results_test/neurons/modelxx/foldxx/runxx',
                 out_dir_glia: str = 'results_test/glia/modelxx/foldxx/runxx',
                 batch_size: int = 32,
                 var_thres: float = 0.0,
                 fold: int = 0,
                celltype: int = -1):
        super().__init__()
        self.PSI_neur = PSI_neur
        self.PSI_glia = PSI_glia
        self.RBP = RBP
        self.batch_size = batch_size
        self.fold = fold
        self.out_dir_neur = out_dir_neur
        self.out_dir_glia = out_dir_glia
        self.var_thres = var_thres
        self.celltype = celltype
        
    def prepare_data(self):
        
        Path(self.out_dir_neur).mkdir(parents=True, exist_ok=True)
        Path(self.out_dir_glia).mkdir(parents=True, exist_ok=True)
                
        # Only keep exons which have a PSI value for both glia and neurons
        PSI = pd.concat((PSI_neur, PSI_glia), axis=1)
        
        split = np.array(pd.DataFrame(PSI_neur.index)[0].str.split('_', expand=True, n=5))
        genes_all = split[:,3]
                
        self.PSI = PSI
        self.RBP = RBP        
        
        # Split in 10 folds and pick fold of interest
        gkf = GroupKFold(n_splits=10)
        trainval_test_indices = list(gkf.split(self.RBP, self.PSI, genes_all))
        trainval_indices, test_idx = trainval_test_indices[self.fold]
        
        gkf2 = GroupKFold(n_splits=9)
        train_val_indices = list(gkf2.split(self.RBP.iloc[trainval_indices],
                                          self.PSI.iloc[trainval_indices],
                                          genes_all[trainval_indices]))
        train_indices, val_indices = train_val_indices[0]
        train_idx = trainval_indices[train_indices]
        val_idx = trainval_indices[val_indices]
        
        # Only keep exons with at least some RBP counts
        idx_tokeep =  np.where(np.sum(RBP, axis=1) > 0)[0]
        train_idx = np.intersect1d(train_idx, idx_tokeep)
        val_idx = np.intersect1d(val_idx, idx_tokeep)
        test_idx = np.intersect1d(test_idx, idx_tokeep)
        
        # Filter for variable exons if needed
        if self.var_thres > 0:
            tokeep = np.where(np.abs(PSI.values[:,0] - PSI.values[:,1]) > self.var_thres)[0]
            self.train_idx = np.intersect1d(train_idx, tokeep)
            self.val_idx = np.intersect1d(val_idx, tokeep)
        else:
            self.train_idx = train_idx
            self.val_idx = val_idx
        
        self.test_idx = test_idx
        
        print(len(self.train_idx))
        print(len(self.val_idx))
        print(len(self.test_idx))
        
        # Write the file genes.csv, so we know which genes were in which fold
        split = np.zeros((len(self.RBP),1), dtype='<U5')
        split[self.train_idx] = 'train'
        split[self.val_idx] = 'valid'
        split[self.test_idx] = 'test'
        
        #tokeep 
        target = np.reshape(PSI.values[:,0], (-1,1))
        genes = pd.DataFrame(np.hstack((split,target)), 
                             index=PSI.index, columns=np.hstack((['split'], PSI.columns[0])))
        genes = genes[genes['split'] != '']
        genes.to_csv(self.out_dir_neur + '/genes.csv')
        
        target = np.reshape(PSI.values[:,1], (-1,1))
        genes = pd.DataFrame(np.hstack((split,target)), 
                             index=PSI.index, columns=np.hstack((['split'], PSI.columns[1])))
        genes = genes[genes['split'] != '']
        genes.to_csv(self.out_dir_glia + '/genes.csv')

        
    def setup(self, stage: str):
        
        if stage == "fit":
            self.RBP_train = RBP_data(self.PSI, self.RBP, self.train_idx, self.celltype)
            self.RBP_val = RBP_data(self.PSI, self.RBP, self.val_idx, self.celltype)
            
        if stage == "test":
            self.RBP_test = RBP_data(self.PSI, self.RBP, self.test_idx, self.celltype)
            
        if stage == "predict":
            self.RBP_predict = RBP_data(self.PSI, self.RBP, self.test_idx, self.celltype)

    def train_dataloader(self):
        return DataLoader(self.RBP_train, batch_size = self.batch_size,
                          num_workers = 1,
                         shuffle = True)
    
    def val_dataloader(self):
        return DataLoader(self.RBP_val, batch_size = self.batch_size,
                          num_workers = 1,
                         shuffle = False)
    
    def test_dataloader(self):
        return DataLoader(self.RBP_test, batch_size = self.batch_size,
                          num_workers = 1,
                         shuffle = False)
    
    def predict_dataloader(self):
        return DataLoader(self.RBP_predict, batch_size = self.batch_size,
                          num_workers = 1,
                         shuffle = False)
    
    
class RBP_data(Dataset):
    def __init__(self, PSI, RBP, idx, celltype):
        
        if celltype != -1:
            self.PSI = np.reshape(PSI.values[idx,celltype], (-1,1))
        else:
            self.PSI = PSI.values[idx]
        self.RBP = RBP.values[idx]
        self.indices = idx
        
    def __len__(self):
        return len(self.PSI)
    
    def __getitem__(self, index):
        PSI = self.PSI[index].astype(np.float32)
        RBP = self.RBP[index].astype(np.float32)
        ind = self.indices[index].astype(int)
        
        sample = [RBP, PSI, ind]
        return sample


### Function to do the cross-validation

This function trains the model, makes predictions, and save the weights learned.

In [4]:
def train_model_celltype(PSI_neur, PSI_glia, RBP, lr, alpha, l1_ratio, batch_size, var_thres,
                      out_neurons, out_glia, celltype):
    # Celltype = 0 for neurons
    # Celltype = 1 for glia
    
    # Keeps track of the learned weights
    weights = 0
    
    # 10 fold CV
    for i in range(10):

        print(i)

        # 5 runs per fold
        for j in range(5):

            out_neur2 = out_neurons + '/fold' + str(i) + '/run' + str(j)
            out_glia2 = out_glia + '/fold' + str(i) + '/run' + str(j)

            # Initialize data module
            dm = ExonDataModule(PSI_neur=PSI_neur,
                                PSI_glia=PSI_glia,
                                RBP=RBP,
                                batch_size=batch_size, fold=i,
                                out_dir_neur=out_neur2,
                                out_dir_glia=out_glia2,
                                var_thres=var_thres,
                                celltype=celltype)
            
            # Initialize model
            model = ElasticLinear(input_dim = 732,
                                  output_dim = 1,
                                  loss_fn = torch.nn.BCEWithLogitsLoss(),
                                  alpha=alpha, l1_ratio=l1_ratio, lr=lr)
            early_stop_callback = EarlyStopping(monitor="val_loss", 
                                                min_delta=0.00, 
                                                patience=10, verbose=False, mode="min")
            trainer = pl.Trainer(callbacks=[early_stop_callback], 
                                 max_epochs=-1,gpus=1)

            # Train model
            trainer.fit(model, datamodule=dm)

            # Save the weights
            weights += model.linear.weight.detach().numpy()/50

            # Make predictions on test set
            y_pred_true = trainer.predict(model, datamodule=dm)

            # Extract predictions
            y_pred_hip = []
            y_true_hip = []

            for j in range(len(y_pred_true)):

                y_pred_hip.extend(y_pred_true[j][0].detach().numpy())
                y_true_hip.extend(y_pred_true[j][1].detach().numpy())

            y_pred_hip = np.array(y_pred_hip)
            y_true_hip = np.array(y_true_hip)

            # Sigmoid activation function
            y_pred_hip = 1/(1 + np.exp(-y_pred_hip))
            
            if celltype == 0:
                out_dir = out_neur2
            else:
                out_dir = out_glia2

            # Save the results
            genes_df = pd.read_csv(out_dir + '/genes.csv', index_col=0)
            gene_ids = np.array(genes_df.index[genes_df['split'] == 'test'], dtype='S')

            preds_h5 = h5py.File('%s/preds.h5' % out_dir, 'w')
            preds_h5.create_dataset('preds', data=y_pred_hip)
            preds_h5.create_dataset('genes', data=gene_ids)
            preds_h5.close()

            targets_h5 = h5py.File('%s/targets.h5' % out_dir, 'w')
            targets_h5.create_dataset('targets', data=y_true_hip)
            targets_h5.create_dataset('genes', data=gene_ids)
            targets_h5.close()
    
    # Save the weights
    if celltype == 0:
        weights = pd.DataFrame(np.transpose(weights), index=RBP.columns, columns=['Neurons'])
        weights.to_csv(out_neurons + '/weights.csv')
    else:
        weights = pd.DataFrame(np.transpose(weights), index=RBP.columns, columns=['Glia'])
        weights.to_csv(out_glia + '/weights.csv')

### Example on human HPC

In [5]:
glia_dir = '../../Zenodo/Human/HPC/PSI/PSI_glia_norm.csv'
neur_dir = '../../Zenodo/Human/HPC/PSI/PSI_neur_norm.csv'
RBP_dir = '../../Zenodo/Human/HPC/RBP/RBP_peaks.csv'

PSI_glia = pd.read_csv(glia_dir, index_col = 0)
PSI_neur = pd.read_csv(neur_dir, index_col = 0)
RBP = pd.read_csv(RBP_dir, index_col = 0)

to_keep = PSI_glia['0'].notna() & PSI_neur['0'].notna()

PSI_glia = PSI_glia[to_keep]
PSI_neur = PSI_neur[to_keep]
RBP = RBP[to_keep]

lr = 0.5
alpha = 0.001
l1_ratio = 0.005
batch_size = 256

var_thres = 0.25 

out_neurons = 'test_output/HPC/neurons/LR_var025/'
out_glia = 'test_output/HPC/glia/LR_var01/'

In [8]:
RBP

,AATF_start,ABCF1_start,AKAP1_start,APOBEC3C_start,AQR_start,BCCIP_start,BUD13_start,CDC40_start,CPEB4_start,CPSF6_start,...,UTP18_end_overlap,UTP3_end_overlap,WDR3_end_overlap,WDR43_end_overlap,XPO5_end_overlap,YBX3_end_overlap,YWHAG_end_overlap,ZC3H11A_end_overlap,ZNF622_end_overlap,ZRANB2_end_overlap
chr10_100002942_100003023_ENSG00000107554_-,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chr10_1000677_1000868_ENSG00000107937_+,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
chr10_100158984_100159055_ENSG00000107566_-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chr10_100167348_100167406_ENSG00000107566_-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
chr10_100174208_100174281_ENSG00000107566_-,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX_96936374_96936500_ENSG00000147202_+,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chrX_96937233_96937351_ENSG00000147202_+,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chrX_96970817_96970861_ENSG00000147202_+,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chrX_9709633_9709760_ENSG00000101849_+,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Run for neurons
train_model_celltype(PSI_neur, PSI_glia, RBP,
                   lr, alpha, l1_ratio, batch_size, var_thres,
                  out_neurons, out_glia, 0)

# Run for glia
train_model_celltype(PSI_neur, PSI_glia, RBP,
                   lr, alpha, l1_ratio, batch_size, var_thres,
                  out_neurons, out_glia, 1, RBP)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores


0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


1456
177
3737



  | Name    | Type              | Params
----------------------------------------------
0 | loss_fn | BCEWithLogitsLoss | 0     
1 | linear  | Linear            | 733   
2 | sigmoid | Sigmoid           | 0     
----------------------------------------------
733       Trainable params
0         Non-trainable params
733       Total params
0.003     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

C:\Users\lcmmichielsen\Anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:102: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
